
# GPU Numba and CuPy Parallelization of Matrix Multiplication

Similary to the multicore parallelization lab, in this lab we will be using Numba and CuPy to accelerate matrix-matrix multiplications using GPU. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba for CUDA](https://numba.readthedocs.io/en/stable/cuda/index.html)
* [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb)
* [Numba.CUDA by Graham Markell](https://github.com/numba/nvidia-cuda-tutorial)
* [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/)
* [CuPy Basics](https://docs.cupy.dev/en/stable/user_guide/basic.html)

[//]: <> (GEOPHYS 257 Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

<br>

### Exercise 0

* You need to request a *T4* node on the cluster. Don't forget that you need to add **--gres=gpu** to your srun command.
* Reminder: on the *T4* nodes you need to load a different version of Python:
```bash
spack load python@3.10.7
```

* Import every Python module, object, and/or function that you need below.##### The following code is used to request the GPU node on the cluster on the t4 partition and then load the correct version of Python


srun --partition=t4 --gres=gpu:1 --x11 --exclusive --pty /bin/bash

. /home/spack/spack/share/spack/setup-env.sh

spack load gcc@9.5.0

spack compiler find

spack load python@3.10.7

juptyer notebook

### Google Remote Desktop Does Not Work for me

The following code is used to request the GPU node on the cluster on the t4 partition and then load the correct version of Python


1. srun --partition=t4 --gres=gpu:1 --x11 --exclusive --pty /bin/bash
2. . /home/spack/spack/share/spack/setup-env.sh
3. spack load gcc@9.5.0
4. spack compiler find
5. spack load python@3.10.7
6. juptyer notebook

### Sherlock Works for me
The following code is used to request the GPU node on the cluster on the serc partition and then load the correct version of Python

1. sbatch --job-name=jupyter-gpu --partition=serc --gres gpu:2 --mem=64G --time=04:00:00
2. module load cuda/10.1.168
3. module load cudnn/7.6.4
4. module load nccl
5. module load py-numba/0.54.1_py39
6. source activate base
7. jupyter lab --no-browser --port=$PORT

<br>

### Exercise 1: Matrix Transpose

Before we examine matrix-matrix multiplication, we will first write a GPU kernel that transposes a square matrix.  This type of problem is a good introduction into how to use the CUDA threading model. The task for this exercise is to write a Numba CUDA kernel that will transpose a square matrix.

**Before you start**, take a look at the following:
* Read over the following notebook that explanes Numba.CUDA kernels: [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb)
* The first matrix-matrix multiplication code (the one that **doesn't** use shared memory) shown at [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). Understanding this code should give a pretty good idea on how to write the transpose kernel. The matrix-matrix kernel code from the NYU lab is shown below.
```python
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
```

**Tasks for this exercise**
* Write a Numba.CUDA kernel that transpose an $NxN$ square matrix.
* Be sure that the transpose kernel can transpose square matrices with sizes of $N$ as small as $N=2$ and as large as $N=10240$.
* Using shared memory is **not** required.


In [ ]:
import numpy as np
from numba import cuda, jit

# CUDA kernel
@cuda.jit
def mattran(A, B):
    """ Perform the matrix transpose of a NxN square matrix, i.e., A = B^T
    """

    # obatin the index
    xpos, ypos = cuda.grid(2)

    # perform the transpose
    if xpos < A.shape[0] and ypos < A.shape[1]:
        B[ypos, xpos] = A[xpos, ypos]


# Host code
# Initialize the data arrays
# N = 2
N = 10240
A = np.linspace(0, N*N,num=N*N, endpoint=True).reshape(N, N)
d_A = cuda.to_device(A)
d_B = cuda.device_array((N, N))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(np.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(A.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel
mattran[blockspergrid, threadsperblock](d_A, d_B)

# Copy the result back to the host
B = d_B.copy_to_host()

# Check the results using assert
assert np.any(A == B.T)

# Note: no output means the test passed

# Release memory
del d_A, d_B, A, B

<br>

### Exercise 2: Using Numba CUDA to parallelize matrix multiplication:

For this exercise, we will use Numba compiled GPU kernels that calculate matrix-matrix multiplication for square matrices. In particular, we will use a GPU kernel that doesn't used shared memory and compared to a GPU kernel that does use shared-memory. Please use the two kernel codes discussed in the following lab: [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). As you will see in this exercise, learning to use shared-memory (akin to user-controlled cache), can take a lot of practice, so in the next exercise, we examine how well the simple shared-memory kernel from the NYU lab compares to the optimized codes provided by NVIDIA in the CuPy package.

#### The tasks for this exercise:
1. Copy the matrix-matrix kernel codes from the NYU lab. Test them for accurracy against *numpy.dot()* and also compare time runtimes these GPU kernels the numpy.dot() function as well. **Note:** Use [CUDA events](https://numba.readthedocs.io/en/stable/cuda-reference/host.html#events) when timing GPU kernel calls because the driver does not "block" the calling process (for case this is IPython). Insted, the kernel is sent to the GPU to run, and then the process (IPython) immediately continues to it's next bit of code. Contrary to GPU kernel calls, calls to copy data to or from the GPU will block the process. For these cases, the calls can be timed the same way that other Python calls are timed.<br> **For both GPU kernels:**
    - Test with square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 5120$, $N=10240$, and $N=20480$. **Tip**, first make sure you can get the GPU codes to work and that you get correct results by testing with $N_{test}=32$.
    - For each $N$ above, test the multiplication for both dtypes: *dtype=float32* and *dtype=float64*.
    - Calculate and show the error between your functions and the *numpy.dot()* function.
    - Calculate and show the *speedup* (or *slowdown*) of your GPU kernel for each $N$ vs *numpy.dot()*. Be sure to include the array copy times in the "total-gpu-kernel runtime.
    - For each $N$ vs, calculate and show the *speedup* of your GPU kernel using *dtype=float32* vs *dtype=float64*. Be sure to include the array copy times in the "total-gpu-kernel runtime."

<br>

2. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
h_A = np.random.random((N, N)).astype(np.<float-type>)
h_B = np.random.random((N, N)).astype(np.<float-type>)
```
<br>

3. For the device memory:
    - Create **d_A** and **d_B** by copying **h_A** and **h_B** to the GPU, and be sure to time the copies
    - Create **d_C** as device-array that is allocated on the GPU (device) only, and not on the host (**Do Not Copy**)

<br>

4. After the GPU matrix-matrix multiplication kernel finishes, **copy** the the *device-array* **d_C** to the *host-array* **h_C**, and be sure to time this copy.

<br>

5. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the speedup or slowdowns vs numpy as well as float32 vs float64. Remember, that your runtime for the GPU kernel include time to compile the kernel (not much you can do to control this). Futhermore, becasue you have to copy data to and off of the GPU, these copy times should be included in the "total-gpu-kernel runtime."

In [ ]:
import numpy as np
from numba import cuda, float32, float64
from timeit import default_timer as timer
from __future__ import division


# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp


# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul_float32(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp


@cuda.jit
def fast_matmul_float64(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float64)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [ ]:
def test_matmul(N, dtype, benchmark=False):
    ''' Test function for matmul kernel
    '''

    # Initialize the data arrays
    h_A = np.random.random((N, N)).astype(dtype)
    h_B = np.random.random((N, N)).astype(dtype)

    # Create cuda events
    evt_beg = cuda.event()
    evt_end = cuda.event()

    # Begin recording the time
    evt_beg.record()

    # Copy the arrays to the device
    d_A = cuda.to_device(h_A)
    d_B = cuda.to_device(h_B)
    d_C = cuda.device_array((N, N))

    # Configure the blocks
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(h_A.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(h_B.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Start the kernel
    matmul[blockspergrid, threadsperblock](d_A, d_B, d_C)

    # Copy the result back to the host
    h_C = d_C.copy_to_host()

    # End recording the time
    evt_end.record()
    evt_end.synchronize()


    # Perform Numpy solution
    np_beg = timer()
    np_C = np.dot(h_A, h_B)
    np_end = timer()

    # Calculate the time
    time_gpu = evt_beg.elapsed_time(evt_end)
    time_cpu = float(np_end - np_beg) * 1000

    # Calculate the relative error
    error = np.sum(abs(np_C - h_C))/np.sum(abs(np_C))

    # Check the results using assert
    if benchmark:
        assert np.allclose(np_C, h_C), "GPU result is not equal to CPU result!"

    # Free the host memory
    del h_A, h_B, h_C, np_C

    # Free the device memory
    del d_A, d_B, d_C


    # Return GPU and CPU time, and relative errors
    return time_gpu, time_cpu, error

In [ ]:
def test_fast_matmul(N, dtype, benchmark=False):
    ''' Test function for the fast matmul kernel
    '''

    # Initialize the data arrays
    h_A = np.random.random((N, N)).astype(dtype)
    h_B = np.random.random((N, N)).astype(dtype)

    # Create cuda events
    evt_beg = cuda.event()
    evt_end = cuda.event()

    # Begin recording the time
    evt_beg.record()

    # Copy arrays to the device
    d_A = cuda.to_device(h_A)
    d_B = cuda.to_device(h_B)
    d_C = cuda.device_array((N, N), dtype=dtype)

    # Configure the blocks
    threadsperblock = (TPB, TPB)
    blockspergrid_x = int(np.ceil(h_A.shape[0] / threadsperblock[1]))
    blockspergrid_y = int(np.ceil(h_B.shape[1] / threadsperblock[0]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Start the kernel based on data type
    if dtype is np.float32:
        fast_matmul_float32[blockspergrid, threadsperblock](d_A, d_B, d_C)
    elif  dtype is np.float64:
        fast_matmul_float64[blockspergrid, threadsperblock](d_A, d_B, d_C)
    else:
        raise ValueError(f'Not support dtype {dtype}')

    # Copy the result back to the host
    h_C = d_C.copy_to_host()

    # End recording the time
    evt_end.record()
    evt_end.synchronize()

    # Perform Numpy solution
    np_beg = timer()
    np_C = np.dot(h_A, h_B)
    np_end = timer()

    # Calculate the time
    time_gpu = evt_beg.elapsed_time(evt_end)
    time_cpu = float(np_end - np_beg) * 1000

    # Calculate the relative error
    error = np.sum(abs(np_C - h_C))/np.sum(abs(np_C))

    # Check the results using assert
    if benchmark:
        assert np.allclose(np_C, h_C), "GPU result is not equal to CPU result!"

    # Free the host memory
    del h_A, h_B, h_C, np_C

    # Free the device memory
    del d_A, d_B, d_C


    # Return GPU and CPU time, and relative errors
    return time_gpu, time_cpu, error

In [ ]:
_, _, _ = test_matmul(32, np.float32, benchmark=True)
_, _, _ = test_matmul(32, np.float64, benchmark=True)
_, _, _ = test_fast_matmul(32, np.float32, benchmark=True)
_, _, _ = test_fast_matmul(32, np.float64, benchmark=True)

/share/software/user/open/py-numba/0.54.1_py39/lib/python3.9/site-packages/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (4) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/share/software/user/open/py-numba/0.54.1_py39/lib/python3.9/site-packages/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (4) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/share/software/user/open/py-numba/0.54.1_py39/lib/python3.9/site-packages/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (4) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/share/software/user/open/py-numba/0.54.1_py39/lib/python3.9/site-packages/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (4) < 2 * SM count (216) will likely result in GPU under utiliza

### Test the performance with cases when N = 5120, 10240, and 20480 for float32 and float64

In [ ]:
# Loop over the matrix size and data type
for N in [5120, 10240, 20480]:
    for dtype in [np.float32, np.float64]:

        # Perform the tests
        time_gpu, time_cpu, error = test_matmul(N, dtype)

        # Print information
        print('Case: N = {}, dtype = {}:'.format(N, dtype))
        print('  GPU   time: {:.2f} s'.format(time_gpu/1000.0))
        print('  Numpy time: {:.2f} s'.format(time_cpu/1000.0))
        print('  Speed up  : {:.2f} X'.format(time_cpu/time_gpu))
        print('  Rela error: {:.6e}'.format(error))
        print('\n')

Case: N = 5120, dtype = <class 'numpy.float32'>:
  GPU   time: 0.62 s
  Numpy time: 2.80 s
  Speed up  : 4.49 X
  Rela error: 7.592821e-08


Case: N = 5120, dtype = <class 'numpy.float64'>:
  GPU   time: 1.23 s
  Numpy time: 6.40 s
  Speed up  : 5.19 X
  Rela error: 1.647218e-15


Case: N = 10240, dtype = <class 'numpy.float32'>:
  GPU   time: 4.26 s
  Numpy time: 22.38 s
  Speed up  : 5.25 X
  Rela error: 8.166743e-08


Case: N = 10240, dtype = <class 'numpy.float64'>:
  GPU   time: 8.00 s
  Numpy time: 50.86 s
  Speed up  : 6.36 X
  Rela error: 2.326111e-15


Case: N = 20480, dtype = <class 'numpy.float32'>:
  GPU   time: 32.02 s
  Numpy time: 184.21 s
  Speed up  : 5.75 X
  Rela error: 1.036160e-07


Case: N = 20480, dtype = <class 'numpy.float64'>:
  GPU   time: 61.89 s
  Numpy time: 415.26 s
  Speed up  : 6.71 X
  Rela error: 3.289052e-15




### Test the performance with cases when N = 5120, 10240, and 20480 for float32 and float64

In [ ]:
# Loop over the matrix size and data type
for N in [5120, 10240, 20480]:
    for dtype in [np.float32, np.float64]:

        # Perform the tests
        time_gpu, time_cpu, error = test_fast_matmul(N, dtype)

        # Print information
        print('Case: N = {}, dtype = {}:'.format(N, dtype))
        print('  GPU   time: {:.2f} s'.format(time_gpu/1000.0))
        print('  Numpy time: {:.2f} s'.format(time_cpu/1000.0))
        print('  Speed up  : {:.2f} X'.format(time_cpu/time_gpu))
        print('  Rela error: {:.6e}'.format(error))
        print('\n')

Case: N = 5120, dtype = <class 'numpy.float32'>:
  GPU   time: 0.45 s
  Numpy time: 2.87 s
  Speed up  : 6.33 X
  Rela error: 7.261742e-08


Case: N = 5120, dtype = <class 'numpy.float64'>:
  GPU   time: 1.32 s
  Numpy time: 6.39 s
  Speed up  : 4.85 X
  Rela error: 1.646969e-15


Case: N = 10240, dtype = <class 'numpy.float32'>:
  GPU   time: 3.03 s
  Numpy time: 22.67 s
  Speed up  : 7.48 X
  Rela error: 7.864619e-08


Case: N = 10240, dtype = <class 'numpy.float64'>:
  GPU   time: 9.54 s
  Numpy time: 50.97 s
  Speed up  : 5.34 X
  Rela error: 2.326500e-15


Case: N = 20480, dtype = <class 'numpy.float32'>:
  GPU   time: 22.48 s
  Numpy time: 184.80 s
  Speed up  : 8.22 X
  Rela error: 1.012555e-07


Case: N = 20480, dtype = <class 'numpy.float64'>:
  GPU   time: 74.59 s
  Numpy time: 415.80 s
  Speed up  : 5.57 X
  Rela error: 3.288806e-15




**Response**:
From the test cases shown above, I find that the the GPU kernel is faster than the numpy.dot() function in all cases, including N = 5120, 10240 and 20480, and for both date type of float32 and float64. The speedup using GPU compared is about 4~8 times faster than the numpy.dot() function in different cases. The speedup is more obvious when N is large. In calculating the time of the GPU kernel, the array copy times are included in the "total-gpu-kernel runtime." I suspect that the most time of the GPU code is possibly copying data to and off of the GPU. It should be also noted that the I first tested the GPU kernel with N = 32 before running the large cases, to test the correctness of the GPU kernel with **benchmark=True** and also compile the code for the first time. The relative errors for all cases are all very small.

When using the shared memory, the GPU version code is faster than the original GPU version without using the shared memory in most cases. However, for the cases when N = 5120 and the dtype is float64 and N=20480 and the dtype is flat64, the GPU version with shared memory is slower than the original version without using the shared memory. In general, the GPU version with shared memory is faster, which is more obvious when N is large.

In terms of the data type, the computation on the float32 is faster than the computation on the float64, and the latter takes double computation time of the former in most cases. This is because the float32 takes less memory than the float64, and the computation on the float32 is faster than the computation on the float64.

<br>

### Exercise 3: CuPy

For this exercise, we will repeat what we did in *Exercise 2*. However, we will use *CuPy* functions, which are similar to *Numpy* funcstions with some added functions for copying data to-the-device-from-the-host and to-the-host-from-the-device. By using CuPy, we can depend on code that has been optimized for the GPU by NVIDIA, and instead of tyring to optimize our matrix-matrix multiplication kernels, we can use a built-in function to calculate the multiplication instead (i.e. [cupy.dot()](https://docs.cupy.dev/en/stable/reference/generated/cupy.dot.html#cupy.dot)).

**Tasks for this exercise:**
* Same as those listed in *Exercise 2*, but compare *cupy.dot()* to *numpy.dot()*.
* Also, reuse the host-arrays, *h_A* and *h_B* above. You will need to call the appropriate *CuPy* fuctions to copy these arrays to the GPU and to copy the result back to the host. You will **not** need to declare the deive-C array before calling *cupy.dot()* because the function will do it for you (like numpy does).


In [ ]:
!which nvcc
!pip install cupy-cuda11x

/share/software/user/open/cuda/11.5.0/bin/nvcc
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


## Test the performance of the matrix multiplication using CuPy

In [ ]:
import cupy as cp

benchmark = False

# loop over the matrix size and data type
for N in [5120, 10240, 20480]:
    for dtype in [np.float32, np.float64]:

        # Initialize the data arrays
        h_A = np.random.random((N, N)).astype(dtype)
        h_B = np.random.random((N, N)).astype(dtype)

        cp_beg = timer()

        # Copy the NumPy array to a Cupy array
        d_A = cp.asarray(h_A)
        d_B = cp.asarray(h_B)

        # Multiply the two matrices using Cupy's dot() function
        cp_C = cp.dot(d_A, d_B)

        # Copy the Cupy array to a NumPy array
        cp_C = cp_C.get()

        cp_end = timer()

        # Perform Numpy solution using numpy array
        np_beg = timer()
        np_C = np.dot(h_A, h_B)
        np_end = timer()

        # Calculate the time
        time_cp = float(cp_end - cp_beg) * 1000
        time_np = float(np_end - np_beg) * 1000

        # Calculate the relative error
        error = np.sum(abs(np_C - cp_C))/np.sum(abs(np_C))

        # Check the results using assert
        if benchmark:
            assert np.allclose(np_C, cp_C), "Cupy result is not equal to Numpy result!"

        # Free the host memory
        del h_A, h_B, d_A, d_B, cp_C, np_C

        # Print information
        print('Case: N = {}, dtype = {}:'.format(N, dtype))
        print('  Cupy  time: {:.4f} s'.format(time_cp/1000.0))
        print('  Numpy time: {:.4f} s'.format(time_np/1000.0))
        print('  Speed up  : {:.2f} X'.format(time_np/time_cp))
        print('  Rela error: {:.6e}'.format(error))
        print('\n')

Case: N = 5120, dtype = <class 'numpy.float32'>:
  Cupy  time: 0.1542 s
  Numpy time: 2.8213 s
  Speed up  : 18.30 X
  Rela error: 8.850817e-07


Case: N = 5120, dtype = <class 'numpy.float64'>:
  Cupy  time: 0.2846 s
  Numpy time: 6.4284 s
  Speed up  : 22.58 X
  Rela error: 1.646527e-15


Case: N = 10240, dtype = <class 'numpy.float32'>:
  Cupy  time: 0.6831 s
  Numpy time: 22.3848 s
  Speed up  : 32.77 X
  Rela error: 3.234979e-07


Case: N = 10240, dtype = <class 'numpy.float64'>:
  Cupy  time: 1.1470 s
  Numpy time: 51.2402 s
  Speed up  : 44.67 X
  Rela error: 2.326129e-15


Case: N = 20480, dtype = <class 'numpy.float32'>:
  Cupy  time: 2.9229 s
  Numpy time: 187.1632 s
  Speed up  : 64.03 X
  Rela error: 1.767918e-06


Case: N = 20480, dtype = <class 'numpy.float64'>:
  Cupy  time: 5.0136 s
  Numpy time: 423.6306 s
  Speed up  : 84.50 X
  Rela error: 3.288878e-15




**Reponse**: In the tests above, I compare the results of the matrix multiplication using Numpy and CuPy. Their relative error is also very small, which is less than 1e-6. The results show that the matrix multiplication using CuPy is much faster than the matrix multiplication using Numpy. Besides, the cp.dot() is much faster then the self-implemented kernels in exercise 2.

<br>

### Exercise $\mathbf{\pi}$: CuPy Interoperability

Numba and CuPy device arrays (GPU arrays) can be accept each other's arrays. See [Interoperability](https://docs.cupy.dev/en/stable/user_guide/interoperability.html).

**Tasks for this exercise**
* Use the **device** arrays, **d_A** and **d_B**, that were created in *Exercise 2* to calculate the matrix-matrix multiplcation using *cupy.dot()*.
* Verify that you get the same results as you did in *Exercise 3*.
* You will need to "wrap" the device arrays before passing them to *cupy.dot()*. Read the *Interoperability* documentation linked above.
    - Time how long it takes (runtime) to "wrap" these arrays.
    - Compare this runtime to the runtime it took to create the device arrays in *Exercise 3*.
    - Provide a quick comment your thoughts on the runtime differences compared above.

## Test Cupy interoperability with Numba

In [ ]:
import cupy as cp
import numpy as np
from numba import cuda, float32, float64
from timeit import default_timer as timer

benchmark = False

# Loop over the matrix size and data type
for N in [5120, 10240 , 20480]:
    for dtype in [np.float32, np.float64]:

        # Initialize the data arrays
        h_A = np.random.random((N, N)).astype(dtype)
        h_B = np.random.random((N, N)).astype(dtype)

        # Copy the arrays to the device
        cuda_beg = timer()
        d_A = cuda.to_device(h_A)
        d_B = cuda.to_device(h_B)
        d_C = cuda.device_array((N, N))
        cuda_end = timer()

        # Wrap the arrays with Cupy
        cupy_beg = timer()
        cp_A = cp.asarray(d_A)
        cp_B = cp.asarray(d_B)
        cupy_end = timer()

        # Perform matrix multiplication using Cupy
        cp_C = cp.dot(cp_A, cp_B)

        # Copy the Cupy array to a NumPy array
        cp_C = cp_C.get()

        # Perform Numpy solution using numpy array
        np_C = np.dot(h_A, h_B)

        # Calculate the time
        time_cuda = float(cuda_end - cuda_beg) * 1000
        time_cupy = float(cupy_end - cupy_beg) * 1000

        # Calculate the relative error
        error = np.sum(abs(np_C - cp_C))/np.sum(abs(np_C))

        # Check the results using assert
        if benchmark:
            assert np.allclose(np_C, cp_C), "Cupy result is not equal to Numpy result!"

        # Free the host memory
        del h_A, h_B, d_A, d_B, cp_C, np_C

        # Print information
        print('Case: N = {}, dtype = {}:'.format(N, dtype, ))
        print('  Cuda wrap time: {:.4f} s'.format(time_cuda/1000.0))
        print('  Cupy wrap time: {:.4f} s'.format(time_cupy/1000.0))
        print('  Speed up      : {:.2f} X'.format(time_cuda/time_cupy))
        print('  Rela error    : {:.6e}'.format(error))
        print('\n')


Case: N = 5120, dtype = <class 'numpy.float32'>:
  Cuda wrap time: 0.0427 s
  Cupy wrap time: 0.0703 s
  Speed up      : 0.61 X
  Rela error    : 8.853206e-07


Case: N = 5120, dtype = <class 'numpy.float64'>:
  Cuda wrap time: 0.0358 s
  Cupy wrap time: 0.0003 s
  Speed up      : 113.03 X
  Rela error    : 1.646544e-15


Case: N = 10240, dtype = <class 'numpy.float32'>:
  Cuda wrap time: 0.0682 s
  Cupy wrap time: 0.0002 s
  Speed up      : 280.29 X
  Rela error    : 3.234823e-07


Case: N = 10240, dtype = <class 'numpy.float64'>:
  Cuda wrap time: 0.1288 s
  Cupy wrap time: 0.0050 s
  Speed up      : 26.02 X
  Rela error    : 2.326191e-15


Case: N = 20480, dtype = <class 'numpy.float32'>:
  Cuda wrap time: 0.2464 s
  Cupy wrap time: 0.0002 s
  Speed up      : 1130.01 X
  Rela error    : 1.767986e-06


Case: N = 20480, dtype = <class 'numpy.float64'>:
  Cuda wrap time: 0.4977 s
  Cupy wrap time: 0.0125 s
  Speed up      : 39.72 X
  Rela error    : 3.288771e-15




**Response**: The results show that the matrix wrap time from numpy array to the device array is larger than that from the drive array to the cupy array. The reason for that, I think, is that the former needs to copy the data from the host to the device, while the latter only needs to copy the data only on the device side. Also, I verified that I get the correct results as I did in Exercise 3 using the assert clause to compare the results of the matrix multiplication using Numpy and CuPy.

In [ ]:
!nvidia-smi

Sun Mar 12 00:15:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   29C    P0    61W / 350W |  16693MiB / 40960MiB |      0%   E. Process |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:C7:00.0 Off |                    0 |
| N/A   

In [ ]:
# The end of the notebook!